<a href="https://colab.research.google.com/github/OculusMode/ILL/blob/main/experimental/Subspaces_with_ILL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from numpy import random
from numpy import linalg as LA

In [90]:
N = 10
M = 4
def get_random_bases(N):
  while True:
    random_matrix = np.random.random((N, N))
    random_basis, _ = LA.qr(random_matrix)
    # making sure we got 10 basis
    if LA.matrix_rank(random_basis) == N:
      break
  return random_basis

def get_random_vector(N, is_normalized = True):
  v = random.random(N)
  return v/LA.norm(v)

# get_random_bases(10)
# get_random_vector(10)

In [123]:
def get_random_subspace_indexes(N, M):
  # putting N-2 since we want maximum number as N-1
  # taking number M-1 times since we will use last left out portion for last partition
  split_points = np.random.choice(N - 2, M - 1, replace=False) + 1
  split_points.sort()
  data = np.arange(N)
  random.shuffle(data)
  random_subspace_indexes = np.split(data, split_points)
  return random_subspace_indexes

def make_partition(basis_matrix, N, M):
  # given an orthonormal basis matrix, give M random subpaces which have no basis in common
  indexes = get_random_subspace_indexes(N, M)
  # return [(basis_matrix[i], i) for i in indexes]
  return tuple(basis_matrix[i] for i in indexes), indexes

def make_random_partitions(basis_matrix, total_partitions, N):
  mean = N//2
  std = N/8
  # I am using normal distribution for this, since we will have more possibility on the middle side.
  s = np.rint(random.normal(mean, std, total_partitions))
  """
  # WE CAN DO THIS TO SEE OUR DISTRIBUTION OVER HISTOGRAM
  import matplotlib.pyplot as plt
  num_bins = N//2
  n, bins, patches = plt.hist(s, num_bins)
  plt.show()
  """
  for i in s:
    if 0<i<N:
      print(make_partition(basis_matrix, N, M=int(i))[1])

random_basis = get_random_bases(N)
# make_partition(random_basis, N, M)
make_random_partitions(random_basis, 100, 10)

[array([5, 2, 3]), array([1]), array([0]), array([4, 7, 9]), array([6, 8])]
[array([9]), array([2]), array([3, 7, 8]), array([0, 4, 6]), array([5, 1])]
[array([3]), array([2, 0, 4, 5, 1]), array([6]), array([9]), array([7, 8])]
[array([1]), array([4]), array([5]), array([7]), array([9, 8]), array([6]), array([0]), array([3, 2])]
[array([7]), array([2]), array([3, 9, 1, 6, 8, 5, 4, 0])]
[array([3]), array([6, 2]), array([8, 7]), array([4, 0, 9, 1, 5])]
[array([5, 3, 8, 0]), array([4, 2]), array([6]), array([1]), array([7, 9])]
[array([4, 9, 0]), array([8]), array([7, 3]), array([5]), array([2, 1, 6])]
[array([9]), array([5]), array([8]), array([0]), array([2, 3, 6, 1, 4, 7])]
[array([9, 1]), array([5, 7]), array([4, 3]), array([6, 8]), array([0, 2])]
[array([7]), array([6, 5]), array([4, 0]), array([9]), array([1]), array([2, 3, 8])]
[array([1, 3, 5]), array([0]), array([2, 9]), array([8]), array([4, 6, 7])]
[array([7]), array([2, 5]), array([3]), array([1, 9]), array([8, 4, 0, 6])]
[ar

100


array([0.15335178, 0.73141882, 0.92942799, 0.46085146, 0.54778756,
       0.59132502])